<a target="_blank" href="https://colab.research.google.com/github/holmrenser/deep_learning/blob/main/word_embeddings.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Word embeddings
In this notebook we work with pretrained word embedding scores from the [GloVe project](https://nlp.stanford.edu/projects/glove/). We use the smallest version, which maps 400,000 words into 50D embedding space, and was trained on 6billion words.

In [1]:
!wget https://github.com/holmrenser/deep_learning/raw/main/data/glove.6B.50d.txt.gz
!gunzip glove.6B.50d.txt.gz

--2024-03-12 15:23:53--  https://github.com/holmrenser/deep_learning/raw/main/data/glove.6B.50d.txt.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/holmrenser/deep_learning/main/data/glove.6B.50d.txt.gz [following]
--2024-03-12 15:23:53--  https://raw.githubusercontent.com/holmrenser/deep_learning/main/data/glove.6B.50d.txt.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69239637 (66M) [application/octet-stream]
Saving to: ‘glove.6B.50d.txt.gz’

glove.6B.50d.txt.gz 100%[===================>]  66,03M  25,4MB/s    in 2,6s    

2024-03-12 15:24:03 (25,4 MB/s) - ‘glove.6B.50d.txt.gz’ saved [69

In [152]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from warnings import warn

In [147]:
class WordEmbedding(nn.Module):
    def __init__(self, vocab: dict[str, int], embeddings: torch.tensor):
        super().__init__()
        self.vocab = vocab
        self.embeddings = nn.Embedding.from_pretrained(embeddings)

    @classmethod
    def from_pretrained(cls, filename: str) -> 'WordEmbedding':
        vocab = {'<pad>':0, '<unk>':1}
        embeddings = []
        
        with open(filename,'r') as fh:
            for i,line in enumerate(fh):
                parts = line.split()
                
                token = parts[0]
                vocab[token] = i + 2 # add two to account for predefined tokens
                
                embedding = list(map(float, parts[1:]))
                embeddings.append(embedding)

        embeddings = torch.tensor(embeddings)
        unk_emb = torch.zeros((1, embeddings.shape[1]))
        pad_emb = embeddings.mean(dim=0)
        embeddings = torch.vstack([pad_emb, unk_emb, embeddings])
        
        return cls(vocab, embeddings)

    def forward(self, word: str) -> torch.tensor:
        i = self.vocab.get(word, 0)
        if i == 0:
            warn(f'{word} is not in the vocabulary, returning average embedding')
        return self.embeddings(torch.tensor([i]))

    def find_closest(self, vec: torch.tensor, top_k: int=1) -> str:
        cos_sim = F.cosine_similarity(emb.embeddings.weight, vec)
        closest_idx = {*torch.argsort(cos_sim)[:top_k]}
        words = [word for word,idx in self.vocab.items() if idx in closest_idx]
        return word[0]

emb = WordEmbedding.from_pretrained('glove.6B.50d.txt')
emb.find_closest(emb('paris') - emb('france') + emb('italy'))

'rome'

In [149]:
emb.find_closest(emb('mud') - emb('water'))

'aisenbergs'

In [153]:
emb('hot1')

/var/folders/8s/ymk92z8x4cb3wb1qbzb0k5v00000gn/T/ipykernel_13780/347095768.py:32: UserWarning: hot1 is not in the vocabulary, returning average embedding
  warn(f'{word} is not in the vocabulary, returning average embedding')


tensor([[-0.1292, -0.2887, -0.0122, -0.0568, -0.2021, -0.0839,  0.3336,  0.1605,
          0.0387,  0.1783,  0.0470, -0.0029,  0.2910,  0.0461, -0.2092, -0.0661,
         -0.0682,  0.0767,  0.3134,  0.1785, -0.1226, -0.0992, -0.0750,  0.0641,
          0.1444,  0.6089,  0.1746,  0.0534, -0.0127,  0.0347, -0.8124, -0.0469,
          0.2019,  0.2031, -0.0394,  0.0697, -0.0155, -0.0341, -0.0653,  0.1225,
          0.1399, -0.1745, -0.0801,  0.0850, -0.0104, -0.1370,  0.2013,  0.1007,
          0.0065,  0.0169]])